## Heat Transfer from Knee to Ice Water

### Assumptions and declarations

#### Declarations

* $t$ is the time variable.
* $T_{w0}$ is the initial temperature of the ice water.
* $T_{sleeve}$ is the temperature of the plastic sleeve and starts at $25 C$
* $T_w$ as the temperature of water flowing throughout the device.

#### Assumptions
* We assume the water has initial temperature of $0 C$
* We assume that the fresh blood perfusing the tissue underneath the skin is constant at temperature $37 C$ and the skin starts at same temperature.
* We assume that the surface area of the capillary bed is the same as the surface area of the knee as a whole (i.e. that fresh blood is constantly being in contact with the tissue.
* We assume that heat transfer from the knee skin to the environment is negligible.
* We assume that the motor contributes negligible.
* We assume that the sleeve material will start at the same temperature as the ice water.





## Known Values
* The device sleeve is a low density polyethylene with thermal coefficient of $0.33 j/(m * K * s)$ at $25 C$
* The skin has a thermal conductivity of $0.31 j/(m * K *s)$
* The water has a thermal conductivity of $0.608 j / (m * k * s)$
* The sleeve of the device is of size $0.079 m^2$ ($10.85" x 11.34"$)



## Key values to figure out

* We don't need to find out how much water is in the sleeve because we assume it is constant temp throughout.
* We don't know the mass of the plastic interface. Note, this needs to be divided in half, since this is double sided plastic sleeve
* We need to figure out how thick the sleeve plastic is between the skin and the sleeve. I'm using the filler variable of .5 mm.
* I am going to use a generic figure for skin depth (2 mm), but I'm not sure that applies to knee skin (https://www.cyh.com/HealthTopics/library/skin.pdf)

## Derivation of the Heat Transfer Equation from Water Through Plastic Sleeve

### Rate of thermal heat transfer:

$\\\dot Q = k * A * \Delta T / d$

$\\\dot Q = k * 0.079 m^2 * (T_w - T_{sleeve})$

### Heat Capacity of Plastic Sleeve:
$\\ \Delta Q = m c \Delta T$

$\\ \Delta Q = m c \Delta T$

$\\ \Delta Q = m c (T_f - T_i)$

$\\ \Delta Q + m c (T_i) =   m c T_f$

$\\ \Delta Q + m c (T_i) / (m c) = T_f$

$\\ \Delta Q = m 0.44 (T_{sleeve}(t_f) - T_{sleeve}(t_i))$

### Change in temp of plastic sleeve as function of time, assuming that $\dot Q$ is constant at instance
Assumption holds for small time periods when the T_{sleeve} has only changed slightly over time

$\\\dot Q \approx  \Delta Q / 1 sec $


$k * 0.079 * (T_w (t) - T_{sleeve}(t_i)) = m 0.44 (T_{sleeve}(t_f) - T_{sleeve}(t_i))$

Let's start with the maximum change in Q over time, which would be when the water is at temperature $0 C$ and the sleeve is at temperature $25 C$
$k * 0.079 * (0 - 25) = m 0.44 (T_{sleeve}(t_f) - y))$


In [5]:
# set up dependencies
import matplotlib.pyplot as plt

In [8]:
#define constants (we can change later if we were wrong)
K_ldpe = 0.33
A_sleeve = 0.079
T_wi = 273 #Kelvin
T_sleevei = 298 #Kelvin
c_ldpe = 2100 #http://www.goodfellow.com/E/Polyethylene-Low-Density.html
d_skin = 0.002 #meters
d_sleeve = 0.0005 #meters
m_sleeve = 0.010 # kg

In [22]:
# Create functions which implements the thermal heat transfer equations and heat capacity equations
def heat_transfer_Q(k, A, d, deltaT):
    return k * A * deltaT / d
    
def heat_capacity_T_final(delta_Q, t_i, c, m):
    return ((m * c * t_i) + delta_Q) / (m * c)

In [27]:
import pandas as pd
from addict import Dict
# Create a quick and dirty definition for defining thermal relationships between objects, ThermItem (ThermodynamicItem)
class ThermItem(object):
    def __init__(self, T_i, c, k, m, name=None):
        self.name = name
        self.t = 0
        self.T = pd.Series() # a series object to hold temperature data over time
        self.T[0] = T_i
        self.deltaQ = Dict() #holds heat transfers over time
        self.m = m
        self.k = k
        self.c = c
        self.relationships = []
        return None
    
    def addRelationship(self, otherItem):
        #Connects two ThermodynamicItems (i.e. we will go ahead and change temp of one object with respect to another)
        self.relationships.add(otherItem)
        otherItem.addRelationship(self)
        
    def updateTemp(deltaT):
        

In [28]:
ThermItem()